In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import os
import keras
import tensorflow as tf 
print(tf.__version__)
print(keras.__version__)
print(pd.__version__)
print(np.__version__)
import re
import glob
# Script Imports
from nn_globals import *
from nn_plotting import gaus, fit_gaus, corr_plot
from dataset import muon_data_split
from nn_evaluate import huber_loss
from nn_training import train_model, lr_schedule
# Keras/TF import
from keras.models import Model, load_model
from keras.regularizers import L1L2
from tensorflow.keras.optimizers import Adam
from keras.layers import Input, BatchNormalization, Dense, Activation
from keras.callbacks import LearningRateScheduler, TerminateOnNaN, ModelCheckpoint

2.6.0
2.6.0
1.3.4
1.19.5


In [2]:
# Import muon data
# 'x' is the array of input variables, 'y' is the q/pT
x_train_displ, x_test_displ, y_train_displ, y_test_displ, dxy_train_displ, dxy_test_displ= muon_data_split(filename=DATAFILEPATH, 
                                                                                                           reg_pt_scale=REG_PT_SCALE, 
                                                                                                           reg_dxy_scale=REG_DXY_SCALE, 
                                                                                                           test_size=TEST_SIZE,
                                                                                                           nvariables = NVARIABLES,
                                                                                                           nentries= NENTRIES,
                                                                                                           batch_size = 128)


[INFO    ] Loading muon data from ./data/NN_input_params_FlatXYZ.npz ...
[INFO    ] Loaded the variables with shape (19300000, 25)
[INFO    ] Loaded the parameters with shape (19300000, 6)
[INFO    ] Loaded the encoded variables with shape (3284620, 23)
[INFO    ] Loaded the encoded parameters with shape (3284620,)
[INFO    ] Loaded # of training and testing events: (2249964, 1034656)
[WARNING ] The last batch for training could be too few! (2024967%128)=7. Please change test_size.
[WARNING ] Try this formula: int(int(3284620*0.685)*0.9) % 128
[WARNING ] The last batch for training after mixing could be too few! (4049935%128)=15. Please change test_size.
[WARNING ] Try this formula: int(int(3284620*0.685)*2*0.9) % 128


In [3]:
def create_model(nvariables, lr=0.001, clipnorm=10., initializer = "glorot_uniform",
                nodes1=64, nodes2=32, nodes3=16, outnodes=2,
                l1_reg = 0.0, l2_reg = 0.0):
  
    regularizer = L1L2(l1=l1_reg, l2=l2_reg)
    bn_momentum = 0.9
    eps = 1e-4

    x = x_in = Input((nvariables,))
    x = BatchNormalization(epsilon=eps, momentum=bn_momentum,name="bn-input")(x)
    
    x = Dense(nodes1, 
               kernel_initializer=initializer,
               use_bias = False,
               kernel_regularizer = regularizer,
               name="hidden-dense-1")(x)
    x = BatchNormalization(epsilon = eps, momentum  = bn_momentum, name = "bn-1")(x)
    x = Activation(activation = "tanh",name="act_1")(x)
    
    if nodes2:
    
        x = Dense(nodes2, 
                   kernel_initializer=initializer,
                   use_bias = False,
                   kernel_regularizer = regularizer,
                   name="hidden-dense-2")(x)
        x = BatchNormalization(epsilon = eps, momentum  = bn_momentum, name = "bn-2")(x)
        x = Activation(activation = "tanh",name="act_2")(x)
        if nodes3:

            x = Dense(nodes3, 
                       kernel_initializer=initializer,
                       kernel_regularizer = regularizer,
                       use_bias = False,
                       name="hidden-dense-3")(x)
            x = BatchNormalization(epsilon = eps, momentum  = bn_momentum, name = "bn-3")(x)
            x = Activation(activation = "tanh", name="act_3")(x)

    x = Dense(outnodes,kernel_initializer = initializer,name="dense-output")(x)
    x = Activation("linear")(x)
    
    model = Model(inputs=x_in, outputs=x,name="baseline-model")
    
    adam = Adam(lr=lr, clipnorm=clipnorm)
    model.compile(optimizer=adam, 
                  loss=huber_loss, 
                  metrics=['acc','mse','mae'])
    model.summary()
    
    return model
    

In [ ]:
assert(keras.backend.backend() == 'tensorflow')

normal_epochs = 300
normal_batch_size = 2000
l1_reg = 0.0
l2_reg = 0.0
learning_rate = 0.0063
gradient_clip_norm = 100.

lr_decay = LearningRateScheduler(lr_schedule, verbose=1)
terminate_on_nan = TerminateOnNaN()
model_training_checkpoint = ModelCheckpoint(monitor="val_loss", 
                                           verbose = 1,
                                           filepath = "checkpoints/model_ckpt_epoch_{epoch:02d}.hdf5",
                                           period = 10)
model = create_model(
                    nvariables = NVARIABLES, 
                    lr = learning_rate, 
                    clipnorm = gradient_clip_norm, 
                    nodes1=10, 
                    nodes2=7, 
                    nodes3=5, 
                    outnodes=2,
                    l1_reg = l1_reg, 
                    l2_reg = l2_reg)

logger.info('Training model with l1_reg: {0} l2_reg: {1}'.format(l1_reg, l2_reg))

model, history = train_model(model, 
                      x_train_displ, 
                      np.column_stack((y_train_displ, dxy_train_displ)),
                      save_model=False, 
                      epochs=normal_epochs, 
                      batch_size=normal_batch_size,
                      callbacks=[lr_decay,terminate_on_nan, model_training_checkpoint], 
                      validation_split=0.1, 
                      verbose=True)

metrics = [len(history.history['loss']), history.history['loss'][-1], history.history['val_loss'][-1]]
logger.info('Epoch {0}/{0} - loss: {1} - val_loss: {2}'.format(*metrics))

2022-01-11 10:52:05.498337: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-01-11 10:52:05.499529: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
/Users/gpradhan/miniforge3/envs/mlp/lib/python3.8/site-packages/keras/optimizer_v2/optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
[INFO    ] Training model with l1_reg: 0.0 l2_reg: 0.0
[INFO    ] Begin training ...


Metal device set to: Apple M1
Model: "baseline-model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 23)]              0         
_________________________________________________________________
bn-input (BatchNormalization (None, 23)                92        
_________________________________________________________________
hidden-dense-1 (Dense)       (None, 10)                230       
_________________________________________________________________
bn-1 (BatchNormalization)    (None, 10)                40        
_________________________________________________________________
act_1 (Activation)           (None, 10)                0         
_________________________________________________________________
hidden-dense-2 (Dense)       (None, 7)                 70        
_________________________________________________________________
bn-2 (BatchNormalizati

2022-01-11 10:52:05.769980: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-01-11 10:52:05.772927: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/300

Epoch 00001: LearningRateScheduler setting learning rate to 0.006300000008195639.


2022-01-11 10:52:06.107368: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


...

1013/1013 [==============================] - 17s 15ms/step - loss: 35.3421 - acc: 0.8949 - mse: 1534.7742 - mae: 26.9357 - val_loss: 27.4625 - val_acc: 0.9151 - val_mse: 1039.4320 - val_mae: 21.0726
Epoch 2/300

Epoch 00002: LearningRateScheduler setting learning rate to 0.006300000008195639.
1013/1013 [==============================] - 16s 16ms/step - loss: 22.2225 - acc: 0.9257 - mse: 706.1531 - mae: 17.1738 - val_loss: 17.7120 - val_acc: 0.9331 - val_mse: 443.9235 - val_mae: 13.8178
Epoch 3/300

Epoch 00003: LearningRateScheduler setting learning rate to 0.006300000008195639.
1013/1013 [==============================] - 16s 15ms/step - loss: 15.4664 - acc: 0.9334 - mse: 338.0123 - mae: 12.1465 - val_loss: 13.6558 - val_acc: 0.9346 - val_mse: 261.7858 - val_mae: 10.7985
Epoch 4/300

Epoch 00004: LearningRateScheduler setting learning rate to 0.006300000008195639.
1013/1013 [==============================] - 15s 14ms/step - loss: 12.8708 - acc: 0.9342 - mse: 232.1196 - mae: 10.2140 - 

1013/1013 [==============================] - 15s 15ms/step - loss: 10.6494 - acc: 0.9356 - mse: 173.0099 - mae: 8.5564 - val_loss: 10.5553 - val_acc: 0.9374 - val_mse: 170.4642 - val_mae: 8.4861
Epoch 30/300

Epoch 00030: LearningRateScheduler setting learning rate to 0.005102999974042177.
1013/1013 [==============================] - 15s 15ms/step - loss: 10.6503 - acc: 0.9355 - mse: 173.0529 - mae: 8.5571 - val_loss: 10.5835 - val_acc: 0.9366 - val_mse: 170.5215 - val_mae: 8.5066

Epoch 00030: saving model to checkpoints/model_ckpt_epoch_30.hdf5
Epoch 31/300

Epoch 00031: LearningRateScheduler setting learning rate to 0.004592699976637959.
1013/1013 [==============================] - 15s 15ms/step - loss: 10.6341 - acc: 0.9356 - mse: 172.7280 - mae: 8.5450 - val_loss: 10.6209 - val_acc: 0.9334 - val_mse: 171.0067 - val_mae: 8.5353
Epoch 32/300

Epoch 00032: LearningRateScheduler setting learning rate to 0.004592699930071831.
1013/1013 [==============================] - 15s 14ms/step -

Epoch 57/300

Epoch 00057: LearningRateScheduler setting learning rate to 0.0037200867664068937.
1013/1013 [==============================] - 15s 14ms/step - loss: 10.5698 - acc: 0.9359 - mse: 171.4992 - mae: 8.4967 - val_loss: 10.4237 - val_acc: 0.9372 - val_mse: 168.4699 - val_mae: 8.3869
Epoch 58/300

Epoch 00058: LearningRateScheduler setting learning rate to 0.0037200867664068937.
1013/1013 [==============================] - 15s 14ms/step - loss: 10.5646 - acc: 0.9359 - mse: 171.4987 - mae: 8.4928 - val_loss: 10.4232 - val_acc: 0.9369 - val_mse: 167.8926 - val_mae: 8.3871
Epoch 59/300

Epoch 00059: LearningRateScheduler setting learning rate to 0.0037200867664068937.
1013/1013 [==============================] - 14s 14ms/step - loss: 10.5689 - acc: 0.9358 - mse: 171.5931 - mae: 8.4961 - val_loss: 10.4127 - val_acc: 0.9374 - val_mse: 168.1478 - val_mae: 8.3790
Epoch 60/300

Epoch 00060: LearningRateScheduler setting learning rate to 0.0037200867664068937.
1013/1013 [================

1013/1013 [==============================] - 15s 14ms/step - loss: 10.5277 - acc: 0.9363 - mse: 170.6924 - mae: 8.4651 - val_loss: 10.3805 - val_acc: 0.9374 - val_mse: 168.3601 - val_mae: 8.3553
Epoch 85/300

Epoch 00085: LearningRateScheduler setting learning rate to 0.002711943117901683.
1013/1013 [==============================] - 15s 14ms/step - loss: 10.5153 - acc: 0.9362 - mse: 170.3293 - mae: 8.4559 - val_loss: 10.3860 - val_acc: 0.9382 - val_mse: 167.8998 - val_mae: 8.3587
Epoch 86/300

Epoch 00086: LearningRateScheduler setting learning rate to 0.002711943117901683.
1013/1013 [==============================] - 14s 14ms/step - loss: 10.5196 - acc: 0.9363 - mse: 170.4790 - mae: 8.4591 - val_loss: 10.3720 - val_acc: 0.9373 - val_mse: 167.8329 - val_mae: 8.3488
Epoch 87/300

Epoch 00087: LearningRateScheduler setting learning rate to 0.002711943117901683.
1013/1013 [==============================] - 15s 15ms/step - loss: 10.5155 - acc: 0.9363 - mse: 170.3794 - mae: 8.4560 - val_lo

1013/1013 [==============================] - 15s 14ms/step - loss: 10.5024 - acc: 0.9364 - mse: 169.9850 - mae: 8.4462 - val_loss: 10.3447 - val_acc: 0.9383 - val_mse: 167.1711 - val_mae: 8.3281
Epoch 112/300

Epoch 00112: LearningRateScheduler setting learning rate to 0.0019770064391195774.
1013/1013 [==============================] - 15s 14ms/step - loss: 10.4945 - acc: 0.9366 - mse: 169.8196 - mae: 8.4402 - val_loss: 10.3483 - val_acc: 0.9375 - val_mse: 166.4565 - val_mae: 8.3316
Epoch 113/300

Epoch 00113: LearningRateScheduler setting learning rate to 0.0019770064391195774.
1013/1013 [==============================] - 15s 14ms/step - loss: 10.4968 - acc: 0.9364 - mse: 169.8620 - mae: 8.4420 - val_loss: 10.3326 - val_acc: 0.9384 - val_mse: 167.0723 - val_mae: 8.3193
Epoch 114/300

Epoch 00114: LearningRateScheduler setting learning rate to 0.0019770064391195774.
1013/1013 [==============================] - 15s 14ms/step - loss: 10.4971 - acc: 0.9364 - mse: 169.7953 - mae: 8.4422 - 